In [37]:
#Chat PDF Script
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import Chroma #vector embeddngs
from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from itertools import count

In [36]:
!pip install chromadb

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ------ --------------------------------- 10.2/67.3 kB ? eta -:--:--
     ----------------- -------------------- 30.7/67.3 kB 330.3 kB/s eta 0:00:01
     ----------------------- -------------- 41.0/67.3 kB 245.8 kB/s eta 0:00:01
     ----------------------- -------------- 41.0/67.3 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 67.3/67.3 kB 304.9 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ------------------------

In [4]:
load_dotenv()

#configuring the api key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [5]:
#Extract the text from the PDF 
def extract_pdf_text(pdf_file):
    text=""
    for pdf in pdf_file:
        pdf_reader= PdfReader(pdf)
        # reads all the pages in the pdf and extracting the texts from the pdf
        for page in pdf_reader.pages:
            text+= page.extract_text()
    return  text

In [31]:
from PyPDF2 import PdfReader

# Extract the text from the PDF file specified by the file path
def extract_pdf_text(pdf_file_path):
    text = ""
    with open(pdf_file_path, "rb") as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        # Read all the pages in the PDF and extract the text from each page
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# Specify the file path to the PDF file
pdf_file_path = r'./resources/the_new_media_in_nigeria.pdf'

# Call the function to extract text from the PDF file
extracted_text = extract_pdf_text(pdf_file_path)

# Print the extracted text
print(extracted_text)


c:\Users\Victor Umunna\AppData\Local\Programs\Python\Python311\Lib\site-packages\PyPDF2\_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(


ANTHONY&PAUL,IMSUJCS4(2) 2020 63–73
63THE NEW MEDIA IN NIGERIA: EVOLUTION, IMPACTS, CHALLENGES AND
PROSPECTS
ANTHONY, I. Igyuve, PhD1
PAUL, U. Obagwu2
1Department of Mass Communication, Nasarawa State University, Keffi
2Department of Mass Communication, Nasarawa State University, Keffi
Corresponding author :ANTHONY I. Igyuve, anthonyigyuve@gmail.com
ABSTRACT
Contemporary societies can no longer be conceived of without the impact of the new media technologies. The
evolution the evolution of new media has brought a radical shift both in media consumption and in the total
ways oflife.These technologies bring the world together in variety of ways that could never have been thought
possible in the years gone by. Anchored on Technological Determinism theory, the paper examined the
evolution of the new media and its implications on s ocio economic development in Nigeria. Findings showed
that,in spite of the plethora of challenges that came with the evolution, especially on the conventional me

In [32]:
#Chunking the text extracted from the PDF
def get_text_chunks(text):
    # Initialize a text splitter object with specified chunk size and overlap
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    
    # Split the input text into smaller chunks using the text splitter object
    chunks = text_splitter.split_text(text)
    
    # Return the list of text chunks
    return chunks

text_chunk = get_text_chunks(extracted_text)


In [33]:
text_chunk

['ANTHONY&PAUL,IMSUJCS4(2) 2020 63–73\n63THE NEW MEDIA IN NIGERIA: EVOLUTION, IMPACTS, CHALLENGES AND\nPROSPECTS\nANTHONY, I. Igyuve, PhD1\nPAUL, U. Obagwu2\n1Department of Mass Communication, Nasarawa State University, Keffi\n2Department of Mass Communication, Nasarawa State University, Keffi\nCorresponding author :ANTHONY I. Igyuve, anthonyigyuve@gmail.com\nABSTRACT\nContemporary societies can no longer be conceived of without the impact of the new media technologies. The\nevolution the evolution of new media has brought a radical shift both in media consumption and in the total\nways oflife.These technologies bring the world together in variety of ways that could never have been thought\npossible in the years gone by. Anchored on Technological Determinism theory, the paper examined the\nevolution of the new media and its implications on s ocio economic development in Nigeria. Findings showed',
 'evolution of the new media and its implications on s ocio economic development in Nigeri

In [40]:
def save_embedding_vector_store(text_chunks):
    embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    db =Chroma.from_texts(text_chunks,embedding=embeddings, persist_directory="./chroma_db")
    db.persist()

In [41]:
text_chunk = save_embedding_vector_store(text_chunk)

In [42]:
def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n{context}?\nQuestion:\n{question}\n\nAnswer:\n
    """

    # Initialize the conversational AI model
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

    # Create a prompt template object
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    # Load the QA chain for generating responses
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

In [ ]:
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    
    #load the indexes from the vector database
    new_db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings, allow_dangerous_deserialization=True)
    #check the similarity with the user question
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)
    
    return response["output_text"]